# 🧠 Módulo 5: Memory Systems - A Memória dos Nossos Bots!

### *"Como fazer seu ChatBot lembrar das conversas anteriores (e não repetir a mesma pergunta 300 vezes!)"*

---

Eaí, pessoal! Tudo tranquilo? 🚀

Tá, mas o que diabos é um **Memory System**? Imagina que você tá conversando com seu amigo no WhatsApp e de repente ele esquece TUDO que vocês falaram 2 segundos atrás. Irritante, né?

É exatamente isso que acontece com os LLMs por padrão - eles são como aquele peixinho Dory do Nemo: sem memória!

Neste módulo vamos aprender:
- ✅ Por que LLMs são "sem memória"
- ✅ Tipos de Memory no LangChain
- ✅ Como implementar cada tipo
- ✅ Quando usar cada um
- ✅ Dicas práticas para produção

**Dica do Pedro**: Memory é o que transforma um bot "burro" em um assistente inteligente!

## 🤔 Por que precisamos de Memory?

Tá, vamos começar com o básico: **LLMs são stateless**!

Isso significa que cada vez que você faz uma pergunta, é como se fosse a primeira vez que você conversa com ele. É tipo assim:

```
Você: "Meu nome é João"
Bot: "Prazer, João!"

Você: "Qual é o meu nome?"
Bot: "Desculpa, não sei seu nome" 🤦‍♂️
```

A **memória** resolve isso mantendo o contexto da conversa. É como dar um caderninho pro bot anotar tudo!

### Analogia do Restaurante 🍕

Imagina um garçom que:
- **Sem memória**: Pergunta seu pedido a cada 2 minutos
- **Com memória**: Lembra do seu pedido e suas preferências

Qual você prefere? Óbvio, né!

In [ ]:
# Bora começar! Primeiro, vamos instalar e importar tudo que precisamos
!pip install langchain langchain-google-genai python-dotenv matplotlib -q

import os
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

# Imports do LangChain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.memory import (
    ConversationBufferMemory,
    ConversationBufferWindowMemory,
    ConversationSummaryMemory,
    ConversationSummaryBufferMemory
)
from langchain.chains import ConversationChain
from langchain.schema import BaseMessage, HumanMessage, AIMessage

print("✅ Bibliotecas carregadas! Bora pro show!")

In [ ]:
# Configurando nossa API do Google (lembra dos módulos anteriores?)
load_dotenv()

# Se não tiver a API key, descomenta a linha abaixo e coloca sua chave
# os.environ["GOOGLE_API_KEY"] = "sua-api-key-aqui"

# Nosso modelo padrão do curso
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    temperature=0.7
)

print("🤖 Modelo configurado! Gemini 2.0 Flash pronto pra ação!")

## 📊 Visualizando o Problema da Memória

Vamos criar um gráfico pra mostrar como funciona a conversa **SEM** memória:

In [ ]:
# Vamos visualizar o problema da falta de memória
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico 1: Sem Memória
conversas = ['Conversa 1', 'Conversa 2', 'Conversa 3', 'Conversa 4']
contexto_sem_memoria = [1, 1, 1, 1]  # Sempre volta ao zero
contexto_com_memoria = [1, 2, 3, 4]  # Acumula conhecimento

ax1.bar(conversas, contexto_sem_memoria, color='red', alpha=0.7)
ax1.set_title('🚫 SEM Memória\n(Cada conversa é isolada)', fontsize=14, fontweight='bold')
ax1.set_ylabel('Contexto Disponível')
ax1.set_ylim(0, 5)

# Gráfico 2: Com Memória
ax2.bar(conversas, contexto_com_memoria, color='green', alpha=0.7)
ax2.set_title('✅ COM Memória\n(Contexto acumulativo)', fontsize=14, fontweight='bold')
ax2.set_ylabel('Contexto Disponível')
ax2.set_ylim(0, 5)

plt.tight_layout()
plt.show()

print("📈 Viu a diferença? Com memória, o bot fica cada vez mais esperto!")

## 🏗️ Arquitetura dos Memory Systems

```mermaid
graph TD
    A[👤 Usuário] --> B[💬 Nova Mensagem]
    B --> C[🧠 Memory System]
    C --> D[📚 Contexto Histórico]
    D --> E[🤖 LLM]
    E --> F[📝 Resposta]
    F --> G[💾 Salvar na Memória]
    G --> C
    F --> A
```

### Como funciona na prática:

1. **Usuário** envia mensagem
2. **Memory System** recupera histórico
3. **LLM** recebe mensagem + contexto
4. **Resposta** é gerada
5. **Tudo é salvo** na memória

**Dica do Pedro**: É tipo um loop infinito de aprendizado!

## 🎯 Tipos de Memory no LangChain

O LangChain tem vários tipos de memória, cada um pra uma situação:

### 1. 🗃️ ConversationBufferMemory
- **O que faz**: Guarda TUDO da conversa
- **Prós**: Contexto completo
- **Contras**: Pode ficar gigante
- **Quando usar**: Conversas curtas

### 2. 🪟 ConversationBufferWindowMemory
- **O que faz**: Guarda só as últimas N mensagens
- **Prós**: Controla o tamanho
- **Contras**: Esquece coisas antigas
- **Quando usar**: Conversas longas com foco no recente

### 3. 📋 ConversationSummaryMemory
- **O que faz**: Resume a conversa
- **Prós**: Muito eficiente
- **Contras**: Pode perder detalhes
- **Quando usar**: Conversas muito longas

### 4. 🔄 ConversationSummaryBufferMemory
- **O que faz**: Mistura resumo + buffer
- **Prós**: Melhor dos dois mundos
- **Contras**: Mais complexo
- **Quando usar**: Conversas longas que precisam de detalhes

**Dica do Pedro**: É como escolher o tamanho da sua mochila pra viajem!

## 📊 Comparando os Tipos de Memory

Vamos criar uma visualização pra entender as diferenças:

In [ ]:
# Comparação visual dos tipos de memória
tipos_memoria = ['Buffer\nCompleto', 'Buffer\nWindow', 'Summary\nOnly', 'Summary+\nBuffer']
uso_tokens = [100, 40, 20, 50]  # Exemplo de uso de tokens
precisao_contexto = [100, 70, 60, 85]  # Precisão do contexto preservado

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico 1: Uso de Tokens
bars1 = ax1.bar(tipos_memoria, uso_tokens, color=['red', 'orange', 'green', 'blue'], alpha=0.7)
ax1.set_title('💰 Uso de Tokens (Custo)', fontsize=14, fontweight='bold')
ax1.set_ylabel('Tokens Utilizados')
ax1.set_ylim(0, 120)

# Adicionando valores nas barras
for bar, valor in zip(bars1, uso_tokens):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, 
             f'{valor}', ha='center', va='bottom', fontweight='bold')

# Gráfico 2: Precisão do Contexto
bars2 = ax2.bar(tipos_memoria, precisao_contexto, color=['red', 'orange', 'green', 'blue'], alpha=0.7)
ax2.set_title('🎯 Precisão do Contexto', fontsize=14, fontweight='bold')
ax2.set_ylabel('Precisão (%)')
ax2.set_ylim(0, 120)

# Adicionando valores nas barras
for bar, valor in zip(bars2, precisao_contexto):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, 
             f'{valor}%', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print("📊 Trade-off clássico: Custo vs Precisão!")
print("💡 A escolha depende do seu caso de uso específico!")

## 🗃️ 1. ConversationBufferMemory - O Arquivo Completo

Vamos começar com o mais simples: guardar TUDO!

É tipo aquela pessoa que guarda todos os WhatsApp desde 2010 😅

In [ ]:
# Criando um ConversationBufferMemory
from langchain.memory import ConversationBufferMemory

# Inicializando a memória
buffer_memory = ConversationBufferMemory(
    return_messages=True,  # Retorna como objetos Message
    memory_key="chat_history"  # Nome da chave no contexto
)

# Criando uma chain com memória
conversation_buffer = ConversationChain(
    llm=llm,
    memory=buffer_memory,
    verbose=True  # Pra ver o que tá rolando nos bastidores
)

print("✅ ConversationBufferMemory criada!")
print("🧠 Esta memória vai guardar TODA a conversa!")

In [ ]:
# Testando o BufferMemory
print("🗣️ Iniciando conversa com BufferMemory...\n")

# Primeira mensagem
resposta1 = conversation_buffer.predict(input="Oi! Meu nome é João e eu adoro pizza.")
print(f"🤖 Bot: {resposta1}\n")

# Segunda mensagem - vamos ver se ele lembra
resposta2 = conversation_buffer.predict(input="Qual é o meu nome e o que eu gosto de comer?")
print(f"🤖 Bot: {resposta2}\n")

# Terceira mensagem - mais contexto
resposta3 = conversation_buffer.predict(input="Eu também trabalho como programador Python.")
print(f"🤖 Bot: {resposta3}\n")

In [ ]:
# Vamos ver o que tem na memória
print("🧠 Conteúdo da memória BufferMemory:")
print("=" * 50)

# Acessando o buffer diretamente
historico = buffer_memory.chat_memory.messages

for i, mensagem in enumerate(historico, 1):
    tipo = "👤 Usuário" if mensagem.type == "human" else "🤖 Bot"
    print(f"{i}. {tipo}: {mensagem.content[:100]}...")

print(f"\n📊 Total de mensagens na memória: {len(historico)}")
print(f"💾 Tamanho aproximado: {sum(len(msg.content) for msg in historico)} caracteres")

## 🪟 2. ConversationBufferWindowMemory - A Janela Deslizante

Agora vamos pro Buffer Window - ele mantém só as últimas N mensagens.

É tipo aquela memória RAM do seu computador: mantém só o que é relevante agora!

In [ ]:
# Criando um ConversationBufferWindowMemory
from langchain.memory import ConversationBufferWindowMemory

# Inicializando com janela de 4 mensagens (2 do usuário + 2 do bot)
window_memory = ConversationBufferWindowMemory(
    k=4,  # Mantém as últimas 4 mensagens
    return_messages=True,
    memory_key="chat_history"
)

# Criando uma nova chain
conversation_window = ConversationChain(
    llm=llm,
    memory=window_memory,
    verbose=True
)

print("✅ ConversationBufferWindowMemory criada!")
print("🪟 Esta memória mantém apenas as últimas 4 mensagens!")

In [ ]:
# Testando o WindowMemory com várias mensagens
print("🗣️ Testando WindowMemory com janela de 4 mensagens...\n")

mensagens_teste = [
    "Oi! Meu nome é Maria.",
    "Eu tenho 25 anos.",
    "Trabalho como designer.",
    "Adoro viajar para a praia.",
    "Tenho um gato chamado Mimi.",
    "Qual é o meu nome mesmo?"  # Vamos ver se ainda lembra!
]

for i, mensagem in enumerate(mensagens_teste, 1):
    print(f"📨 Mensagem {i}: {mensagem}")
    resposta = conversation_window.predict(input=mensagem)
    print(f"🤖 Bot: {resposta}\n")
    
    # Mostrando quantas mensagens estão na memória
    qtd_memorias = len(window_memory.chat_memory.messages)
    print(f"🧠 Mensagens na memória: {qtd_memorias}/4\n")
    print("-" * 60)

## 📋 3. ConversationSummaryMemory - O Resumidor

Agora a coisa fica interessante! O SummaryMemory não guarda as mensagens - ele faz um **resumo**!

É tipo aquele amigo que conta a fofoca resumida: "Ah, rolou um drama, mas o importante é que no final deu tudo certo" 😂

In [ ]:
# Criando um ConversationSummaryMemory
from langchain.memory import ConversationSummaryMemory

# Inicializando - precisa de um LLM pra fazer os resumos!
summary_memory = ConversationSummaryMemory(
    llm=llm,  # Usa o mesmo LLM pra resumir
    return_messages=False,  # Retorna como string
    memory_key="chat_history"
)

# Criando uma nova chain
conversation_summary = ConversationChain(
    llm=llm,
    memory=summary_memory,
    verbose=True
)

print("✅ ConversationSummaryMemory criada!")
print("📋 Esta memória faz resumos automáticos da conversa!")
print("🤖 O próprio LLM vai resumir conforme a conversa cresce!")

In [ ]:
# Testando o SummaryMemory
print("🗣️ Testando SummaryMemory...\n")

# Vamos simular uma conversa longa
mensagens_longas = [
    "Oi! Meu nome é Carlos, tenho 30 anos e sou engenheiro de software.",
    "Trabalho numa startup de fintech aqui em São Paulo há 3 anos.",
    "Estou desenvolvendo um app de investimentos em Python e React.",
    "O app já tem 10.000 usuários e está crescendo 20% ao mês.",
    "Estamos levantando uma rodada de investimento de R$ 2 milhões.",
    "Me fala um resumo de tudo que conversamos até agora?"
]

for i, mensagem in enumerate(mensagens_longas, 1):
    print(f"📨 Mensagem {i}: {mensagem}")
    resposta = conversation_summary.predict(input=mensagem)
    print(f"🤖 Bot: {resposta}\n")
    
    # Vamos espiar o resumo que está sendo mantido
    if i > 2:  # Depois de algumas mensagens
        resumo_atual = summary_memory.buffer
        print(f"📋 Resumo na memória: {resumo_atual[:150]}...\n")
    
    print("-" * 80)

## 🔄 4. ConversationSummaryBufferMemory - O Híbrido

E agora o **melhor dos dois mundos**! O SummaryBufferMemory:

- Mantém as mensagens **recentes** completas
- **Resume** as mensagens antigas

É tipo ter um assistente que anota tudo detalhado no dia, mas faz resumos do mês passado!

In [ ]:
# Criando um ConversationSummaryBufferMemory
from langchain.memory import ConversationSummaryBufferMemory

# Inicializando com limite de tokens
summary_buffer_memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=200,  # Quando passar disso, resume as antigas
    return_messages=True,
    memory_key="chat_history"
)

# Criando uma nova chain
conversation_summary_buffer = ConversationChain(
    llm=llm,
    memory=summary_buffer_memory,
    verbose=True
)

print("✅ ConversationSummaryBufferMemory criada!")
print("🔄 Esta memória combina resumo + buffer!")
print("📊 Limite: 200 tokens - depois disso resume as antigas!")

In [ ]:
# Testando o SummaryBufferMemory
print("🗣️ Testando SummaryBufferMemory...\n")

mensagens_hibridas = [
    "Sou a Ana, médica veterinária de 28 anos.",
    "Trabalho numa clínica veterinária em Belo Horizonte desde 2020.",
    "Tenho especialização em animais exóticos - papagaios, iguanas, etc.",
    "Atendo cerca de 50 animais por semana, a maioria cães e gatos.",
    "Estou pensando em abrir minha própria clínica no ano que vem.",
    "Qual é a minha profissão e onde trabalho?"
]

for i, mensagem in enumerate(mensagens_hibridas, 1):
    print(f"📨 Mensagem {i}: {mensagem}")
    resposta = conversation_summary_buffer.predict(input=mensagem)
    print(f"🤖 Bot: {resposta}\n")
    
    # Verificando o estado da memória
    memoria_vars = summary_buffer_memory.load_memory_variables({})
    print(f"🧠 Estado da memória após mensagem {i}:")
    if hasattr(summary_buffer_memory, 'moving_summary_buffer') and summary_buffer_memory.moving_summary_buffer:
        print(f"📋 Tem resumo: {summary_buffer_memory.moving_summary_buffer[:100]}...")
    else:
        print("📋 Ainda não tem resumo - só mensagens recentes")
    
    print("-" * 70)

## 📊 Comparação Prática dos Memory Types

Vamos fazer um teste prático pra ver como cada tipo se comporta:

In [ ]:
# Função para calcular tamanho aproximado da memória
def calcular_tamanho_memoria(memory_obj):
    """Calcula o tamanho aproximado da memória em caracteres"""
    try:
        # Tenta carregar as variáveis da memória
        vars_memoria = memory_obj.load_memory_variables({})
        
        # Soma todos os caracteres
        tamanho_total = 0
        for key, value in vars_memoria.items():
            if isinstance(value, str):
                tamanho_total += len(value)
            elif isinstance(value, list):
                for item in value:
                    if hasattr(item, 'content'):
                        tamanho_total += len(item.content)
                    else:
                        tamanho_total += len(str(item))
        
        return tamanho_total
    except:
        return 0

# Testando com uma sequência de mensagens
mensagens_comparacao = [
    "Oi, sou o Pedro, desenvolvedore Python de 35 anos.",
    "Trabalho com IA e Machine Learning há 8 anos.",
    "Já criei mais de 100 modelos de ML em produção.",
    "Especialista em LangChain, FastAPI e AWS.",
    "Dou aulas online e já treinei mais de 5000 alunos."
]

# Criando memorias limpas para comparação
memorias_teste = {
    'Buffer': ConversationBufferMemory(return_messages=True),
    'Window': ConversationBufferWindowMemory(k=4, return_messages=True),
    'Summary': ConversationSummaryMemory(llm=llm),
    'SummaryBuffer': ConversationSummaryBufferMemory(llm=llm, max_token_limit=150)
}

# Simulando as mensagens em todas as memórias
for mensagem in mensagens_comparacao:
    for nome, memoria in memorias_teste.items():
        # Simula uma conversa adicionando mensagem do usuário e uma resposta padrão
        memoria.save_context(
            {"input": mensagem}, 
            {"output": f"Entendi! Obrigado por compartilhar isso, {mensagem.split()[2] if len(mensagem.split()) > 2 else 'usuário'}!"}
        )

print("📊 Comparação de tamanhos de memória após 5 interações:")
print("=" * 60)

tamanhos = {}
for nome, memoria in memorias_teste.items():
    tamanho = calcular_tamanho_memoria(memoria)
    tamanhos[nome] = tamanho
    print(f"{nome:15} : {tamanho:4d} caracteres")

print("\n💡 Conclusões:")
print(f"🥇 Mais eficiente: {min(tamanhos, key=tamanhos.get)}")
print(f"📚 Mais completo: {max(tamanhos, key=tamanhos.get)}")

## 📊 Visualização Final - Eficiência vs Completude

In [ ]:
# Gráfico de barras comparativo final
nomes = list(tamanhos.keys())
valores = list(tamanhos.values())
cores = ['red', 'orange', 'green', 'blue']

plt.figure(figsize=(12, 8))

# Gráfico de barras
bars = plt.bar(nomes, valores, color=cores, alpha=0.7, edgecolor='black', linewidth=2)

# Customização
plt.title('📊 Comparação de Uso de Memória\n(Menor = Mais Eficiente)', 
          fontsize=16, fontweight='bold', pad=20)
plt.ylabel('Caracteres Utilizados', fontsize=12, fontweight='bold')
plt.xlabel('Tipos de Memory', fontsize=12, fontweight='bold')

# Adicionando valores nas barras
for bar, valor in zip(bars, valores):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(valores)*0.01, 
             f'{valor}', ha='center', va='bottom', fontweight='bold', fontsize=11)

# Adicionando linha de referência
plt.axhline(y=np.mean(valores), color='red', linestyle='--', alpha=0.7, 
            label=f'Média: {int(np.mean(valores))}')

plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print("📈 Liiindo! Agora você sabe exatamente qual memory usar!")

## 💡 Dicas Práticas do Pedro

### 🎯 Quando usar cada tipo?

| Tipo | Use quando | Exemplo |
|------|------------|--------|
| **Buffer** | Conversas curtas e precisas | Chat de suporte |
| **Window** | Conversas longas, contexto recente | Games, assistentes |
| **Summary** | Conversas muito longas | Terapia, consultoria |
| **SummaryBuffer** | Melhor dos dois mundos | Assistentes corporativos |

### 🚀 Dicas de Performance:

1. **Monitore o tamanho**: Use `len()` na memória
2. **Defina limites**: Sempre coloque `max_token_limit`
3. **Teste em produção**: Cada caso é um caso
4. **Combine tipos**: Use diferentes memórias pra diferentes usuários

### ⚠️ Pegadinhas Comuns:

- **BufferMemory** pode explodir o contexto
- **SummaryMemory** pode perder detalhes importantes
- **WindowMemory** esquece coisas antigas
- Todos custam tokens! 💰

**Dica do Pedro**: Sempre teste com dados reais do seu domínio!

## 🔧 Exercício 1: Implementando Memory Customizada

Bora criar uma memória personalizada que atende às suas necessidades específicas!

**Desafio**: Criar uma memória que:
1. Mantém as últimas 3 mensagens completas
2. Salva informações importantes do usuário (nome, profissão, etc.)
3. Mostra um resumo das informações salvas

In [ ]:
# EXERCÍCIO 1: Implemente uma MemoryCustomizada
class MemoryCustomizada:
    def __init__(self):
        self.mensagens_recentes = []  # Últimas 3 mensagens
        self.info_usuario = {}        # Informações importantes
        self.max_mensagens = 3
    
    def adicionar_mensagem(self, usuario_msg, bot_msg):
        """Adiciona uma nova troca de mensagens"""
        # TODO: Implementar lógica para:
        # 1. Adicionar mensagem à lista
        # 2. Manter apenas as últimas 3
        # 3. Extrair informações importantes (nome, idade, profissão)
        pass
    
    def get_contexto(self):
        """Retorna o contexto atual para o LLM"""
        # TODO: Implementar lógica para:
        # 1. Formatar mensagens recentes
        # 2. Incluir informações do usuário
        # 3. Retornar string formatada
        pass

# Teste sua implementação aqui:
# memory_custom = MemoryCustomizada()
# memory_custom.adicionar_mensagem("Oi, sou João", "Prazer, João!")
# print(memory_custom.get_contexto())

print("🏗️ Seu turno! Implemente a MemoryCustomizada acima")
print("💡 Dica: Use regex ou split() para extrair informações")

## 🎮 Exercício 2: Sistema de Memory Inteligente

**Desafio Avançado**: Criar um sistema que **escolhe automaticamente** o tipo de memória baseado no contexto!

**Regras**:
- Se conversa < 5 mensagens → BufferMemory
- Se conversa entre 5-15 mensagens → WindowMemory
- Se conversa > 15 mensagens → SummaryBufferMemory

In [ ]:
# EXERCÍCIO 2: Sistema Inteligente de Memory
class MemoryInteligente:
    def __init__(self, llm):
        self.llm = llm
        self.contador_mensagens = 0
        self.memory_atual = None
        self.tipo_atual = None
    
    def escolher_memory_tipo(self):
        """Escolhe o tipo de memória baseado no número de mensagens"""
        # TODO: Implementar lógica de escolha
        # Retorna: 'buffer', 'window', ou 'summary_buffer'
        pass
    
    def processar_mensagem(self, input_msg):
        """Processa uma nova mensagem com a memória apropriada"""
        # TODO: Implementar lógica para:
        # 1. Incrementar contador
        # 2. Escolher tipo de memória
        # 3. Migrar dados se necessário
        # 4. Processar mensagem
        pass
    
    def migrar_memoria(self, memoria_antiga, tipo_novo):
        """Migra dados entre diferentes tipos de memória"""
        # TODO: Implementar migração de dados
        pass

# Teste seu sistema aqui:
# memory_smart = MemoryInteligente(llm)
# for i in range(20):
#     resultado = memory_smart.processar_mensagem(f"Mensagem {i+1}")
#     print(f"Mensagem {i+1}: Usando {memory_smart.tipo_atual}")

print("🧠 Desafio avançado! Crie um sistema que se adapta automaticamente")
print("🎯 Objetivo: Otimizar performance baseado no tamanho da conversa")

## 🎯 Casos de Uso Reais

Vamos ver onde cada tipo de memória brilha na vida real:

In [ ]:
# Simulando casos de uso reais
casos_uso = {
    "🏥 Suporte Médico": {
        "memory": "SummaryBuffer",
        "motivo": "Precisa lembrar histórico médico completo + sintomas recentes",
        "exemplo": "Paciente com diabetes menciona novos sintomas"
    },
    "🛒 E-commerce": {
        "memory": "Window",
        "motivo": "Foco nos produtos vistos recentemente",
        "exemplo": "Cliente comparando smartphones nas últimas mensagens"
    },
    "📚 Tutor Educacional": {
        "memory": "Buffer",
        "motivo": "Conversas curtas e focadas em exercícios específicos",
        "exemplo": "Explicando um conceito de matemática"
    },
    "💼 Assistente Executivo": {
        "memory": "SummaryBuffer",
        "motivo": "Precisa lembrar decisões passadas + contexto atual",
        "exemplo": "Acompanhando projeto de 6 meses com reuniões semanais"
    }
}

print("🎯 CASOS DE USO REAIS")
print("=" * 60)

for caso, info in casos_uso.items():
    print(f"\n{caso}")
    print(f"   Memory: {info['memory']}")
    print(f"   Por quê: {info['motivo']}")
    print(f"   Exemplo: {info['exemplo']}")

print("\n💡 A escolha da memória pode fazer ou quebrar seu produto!")

## 🔮 Preparando para os Próximos Módulos

Liiiindo! Agora que dominamos Memory Systems, vamos ver como isso se conecta com o que vem por aí:

### 🗂️ Módulo 6 - Document Loading
- **Conexão**: Vamos usar memória para lembrar de documentos processados
- **Preparação**: Memory vai guardar contexto de múltiplos documentos

### 🔍 Módulo 7 - Vector Store
- **Conexão**: Memory + Vector Store = busca semântica com contexto
- **Preparação**: Combinar memória conversacional com busca em documentos

### 🤖 Módulo 8 - RAG
- **Conexão**: Memory é ESSENCIAL no RAG para conversas contextuais
- **Preparação**: "Lembre do que discutimos sobre este documento"

**Dica do Pedro**: Memory é a base para construir assistentes inteligentes de verdade!

In [ ]:
# Preview do que vem no próximo módulo
print("🔮 PREVIEW - PRÓXIMO MÓDULO")
print("=" * 40)
print("📚 Document Loading & Splitters")
print("")
print("Vamos aprender a:")
print("• Carregar PDFs, Word, websites")
print("• Dividir documentos grandes")
print("• Combinar com Memory Systems")
print("• Preparar dados para RAG")
print("")
print("🚀 Com Memory + Documents = Assistente que lembra E aprende!")

# Salvando um exemplo de memory para usar no próximo módulo
import pickle

# Criando uma memória de exemplo para carregar no próximo módulo
memory_exemplo = ConversationBufferMemory(return_messages=True)
memory_exemplo.save_context(
    {"input": "Vou analisar alguns documentos sobre IA"}, 
    {"output": "Perfeito! Vou lembrar que você está interessado em documentos sobre IA!"}
)

print("💾 Memória de exemplo salva para o próximo módulo!")

## 🎊 Resumo Final - O que Aprendemos

### ✅ Conceitos Dominados:

1. **🧠 Por que Memory é Essencial**
   - LLMs são stateless por padrão
   - Memory transforma bots em assistentes

2. **🗃️ ConversationBufferMemory**
   - Guarda tudo
   - Perfeito para conversas curtas

3. **🪟 ConversationBufferWindowMemory**
   - Mantém últimas N mensagens
   - Controla o tamanho do contexto

4. **📋 ConversationSummaryMemory**
   - Resume conversas automaticamente
   - Muito eficiente em tokens

5. **🔄 ConversationSummaryBufferMemory**
   - Melhor dos dois mundos
   - Buffer recente + resumo antigo

### 🎯 Quando Usar Cada Um:

| Cenário | Memory Ideal | Por quê |
|---------|-------------|--------|
| Chat suporte | Buffer | Conversas curtas e precisas |
| Game/App | Window | Contexto recente mais importante |
| Consultoria | Summary | Conversas muito longas |
| Assistente corporativo | SummaryBuffer | Equilibrio perfeito |

### 🚀 Próximos Passos:
- Módulo 6: Document Loading (como dar "conhecimento" pro bot)
- Módulo 7: Vector Store (busca semântica inteligente) 
- Módulo 8: RAG (combinando tudo!)

**Dica Final do Pedro**: Memory é o primeiro passo para criar assistentes que realmente **entendem** e **lembram**. No próximo módulo vamos dar "conhecimento" pra eles!

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-05_img_01.png)